## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-14-15-50-00 +0000,wsj,"Week Ahead for FX, Bonds: Investors Await U.S....",https://www.wsj.com/economy/week-ahead-for-fx-...
1,2025-11-14-15-49-23 +0000,nypost,Mysterious black ring over Disneyland spooks C...,https://nypost.com/2025/11/14/us-news/mysterio...
2,2025-11-14-15-49-00 +0000,wsj,"Dollar Could Benefit From U.K., French Fiscal ...",https://www.wsj.com/finance/currencies/asian-c...
3,2025-11-14-15-47-00 +0000,wsj,Topgolf in Talks to Be Sold to Private-Equity ...,https://www.wsj.com/business/deals/topgolf-in-...
4,2025-11-14-15-45-47 +0000,bbc,Mass killings probe in Sudan will hold culprit...,https://www.bbc.com/news/articles/c70jd4elgwzo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2373/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,trump,32
79,new,18
13,california,16
161,market,11
60,cut,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
344,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...,103
326,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...,73
268,2025-11-13-21-21-08 +0000,nypost,BBC apologizes to Trump over selectively edite...,https://nypost.com/2025/11/13/us-news/bbc-apol...,70
185,2025-11-14-03-04-50 +0000,nypost,Va. prosecutor accused of ‘stymying the invest...,https://nypost.com/2025/11/13/us-news/va-prose...,68
20,2025-11-14-15-16-18 +0000,nypost,Prosecutor appoints himself to take over Georg...,https://nypost.com/2025/11/14/us-news/prosecut...,68


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
344,103,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...
100,50,2025-11-14-11-24-34 +0000,nypost,Friends of NYC designer Martha Nolan joked abo...,https://nypost.com/2025/11/14/us-news/nyc-desi...
220,50,2025-11-13-23-34-00 +0000,wsj,Boeing Defense Workers Approve New Five-Year C...,https://www.wsj.com/business/boeing-defense-wo...
263,49,2025-11-13-21-46-58 +0000,nypost,Stocks have worst day in a month amid AI conce...,https://nypost.com/2025/11/13/business/stocks-...
326,41,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...
333,39,2025-11-13-17-01-23 +0000,nypost,Rep. Anna Paulina Luna launches probe into wil...,https://nypost.com/2025/11/13/us-news/rep-anna...
268,38,2025-11-13-21-21-08 +0000,nypost,BBC apologizes to Trump over selectively edite...,https://nypost.com/2025/11/13/us-news/bbc-apol...
45,35,2025-11-14-14-21-53 +0000,bbc,Government borrowing costs jump after Reeves d...,https://www.bbc.com/news/articles/cy5qz4qdq95o...
153,32,2025-11-14-09-00-00 +0000,cbc,Health Canada ignored expert advice to expand ...,https://www.cbc.ca/news/canada/opioid-overdose...
99,30,2025-11-14-11-25-00 +0000,wsj,"Tech, Media & Telecom Roundup: Market Talk",https://www.wsj.com/business/tech-media-teleco...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
